In [11]:
# TODO progressapplyからbatch処理に変える

In [2]:
!pip3 install tensorflow_text>=2.0.0rc0

In [12]:
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD

SEED = 29
N_COMPONENTS = 20 # TruncatedSVD
csv_path = "/content/train.csv"
col_name = "C" # USEをする列名

# TODO:ここに他のモデルのpathとかも追加する
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'

# Load Data
# =====================================================
df = pd.read_csv(csv_path, engine='python', encoding='utf-8').head(1000)
print(df.shape)
display(df.head())

(1000, 4)


,id,code,file_extension,label
0,8e7e71745d,a0 = alpha_winder(0)\n ...,py,0
1,f67fb60d2f,"[&](const ExprHandle& m, const ExprHandl...",cpp,0
2,9125d50679,self.play(\n pixels_to_dete...,py,0
3,e03dac0faa,"ksession.execute( new Message( ""help"" ...",java,1
4,ac09060d54,def test_is_fast(self):\n for token...,py,1


In [14]:
# preprocess
# =====================================================
df[['A', 'B', 'C', 'D', 'E']] = df['code'].str.split('\n', expand=True)
display(df.head())

,id,code,file_extension,label,A,B,C,D,E
0,8e7e71745d,a0 = alpha_winder(0)\n ...,py,0,a0 = alpha_winder(0),rebased_winder = lambda alpha:...,colored_line = Line(num_plane....,colored_line.match_style(base_...,if self.use_fancy_lines:
1,f67fb60d2f,"[&](const ExprHandle& m, const ExprHandl...",cpp,0,"[&](const ExprHandle& m, const ExprHandl...",,Tensor* chunk_0 = Compute(,"""chunk"",","{{M, ""m""}, {N / 2, ""n""}},"
2,9125d50679,self.play(\n pixels_to_dete...,py,0,self.play(,"pixels_to_detect.set_fill, WHITE, 1,","submobject_mode = ""lagged_start"",",run_time = 2,)
3,e03dac0faa,"ksession.execute( new Message( ""help"" ...",java,1,"ksession.execute( new Message( ""help"" ...",},,@Test,public void testEvents() throws Exception {
4,ac09060d54,def test_is_fast(self):\n for token...,py,1,def test_is_fast(self):,"for tokenizer, pretrained_name, kwargs...","with self.subTest(""{} ({})"".format...",tokenizer_r = self.rust_tokeni...,tokenizer_p = self.tokenizer_c...


In [15]:
import tensorflow_hub as hub

import numpy as np
import tensorflow_text

# Some texts of different lengths.
# english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]

embedder = hub.load(module_url)
# Compute embeddings.
tqdm.pandas()
features = np.stack(df[col_name].fillna("").progress_apply(lambda x: embedder(x).numpy().reshape(-1)).values)

100%|██████████| 1000/1000 [01:15<00:00, 13.16it/s]


In [16]:
# SVD
# =====================================================
svd = TruncatedSVD(n_components=N_COMPONENTS, random_state=SEED)
svd_features = svd.fit_transform(features)

In [17]:
# save
# =====================================================
np.save("USE",features)
np.save("USE_svd",svd_features)